# Bayesian Variable Selection for Uncorrelated Predictors

Global Imports

In [1]:
import numpy as np
import pymc3 as pm
import theano.tensor as tt

from arviz import summary, plot_trace
from theano.tensor import _shared
from sklearn.preprocessing import StandardScaler

## Uncorrelated Predictors

In [16]:
# Simulate Data
k = 20 # number of predictors
p = 5 # number of nonzero predictors
n = 1000
sigma = 0.5

np.random.seed(615)

zero_idx = np.random.choice(np.arange(0, k), k-p, replace=False)
true_beta = np.random.randn(k)
true_beta[zero_idx] = 0

# Predictor variable
X = pm.MvNormal.dist(mu=np.zeros(k), cov=np.diag(np.ones(k)), shape=[1, k]).random(size=n)

# Simulate outcome variable
y = X.dot(true_beta) + np.random.randn(n) * sigma 

In [17]:
print(true_beta)
np.linalg.inv(X.T@X)@X.T@y

[-1.02976882  0.          0.         -0.22188525  0.44980561  0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          1.05456722  0.          0.
 -1.06592054  0.        ]


array([-1.01714618, -0.01625893, -0.00351084, -0.28006783,  0.44542211,
       -0.01220991, -0.00860135,  0.01057332,  0.0083056 ,  0.0197452 ,
        0.0010527 ,  0.00515986,  0.03849674, -0.01107302,  0.01349936,
        1.04489314, -0.01098878, -0.01893812, -1.04890962, -0.02010866])

In [18]:
scaler_x = StandardScaler()
scaler_y = StandardScaler()
y = scaler_y.fit_transform(y.reshape(-1, 1)).ravel()
X = scaler_x.fit_transform(X)

In [19]:
print(true_beta)
print(np.linalg.inv(X.T@X)@X.T@y)
print(scaler_y.inverse_transform(scaler_x.inverse_transform(np.linalg.inv(X.T@X)@X.T@y)))

[-1.02976882  0.          0.         -0.22188525  0.44980561  0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          1.05456722  0.          0.
 -1.06592054  0.        ]
[-5.26563750e-01 -8.59651521e-03 -1.25043432e-03 -1.43678214e-01
  2.30349046e-01 -6.60513978e-03 -4.37155848e-03  5.71848680e-03
  4.73624392e-03  9.81960623e-03  5.19403435e-04  2.57015908e-03
  2.08522125e-02 -5.58917609e-03  6.99037300e-03  5.34373482e-01
 -6.12025628e-03 -9.97442295e-03 -5.55034609e-01 -1.12577663e-02]
[-1.09385827  0.07551868 -0.07061976 -0.27297455  0.46413591  0.06688127
 -0.00492034 -0.0056113  -0.00249954  0.1095043   0.06013847  0.06652993
  0.06661442  0.02227378  0.01937467  1.02996563  0.09947127  0.02078644
 -1.01952879  0.18149412]


In [ ]:
x = _shared(X)
y_obs = _shared(y)

### Linear Regression

In [ ]:
# Linear Regression
with pm.Model() as lm_model:
    # Priors
    beta = pm.Normal("beta", mu=0, sigma=5, shape=k)
    sd = pm.Gamma("sd", alpha=0.05, beta=0.1, shape=1)
    
    # Likelihood
    mu = pm.Deterministic("mu", tt.dot(X, beta))
    y_ = pm.Normal("y_obs", mu=mu, sd=sd, observed=y)

In [ ]:
model1.check_test_point()

In [ ]:
with model1:
    tr = pm.sample(2000, tune=1000, cores=4)

In [ ]:
lines= [("beta", {"beta_dim_0": i}, true_beta[i]) for i in range(0, k)]
fig1 = plot_trace(tr, var_names=["beta"], lines=lines)

### Spike and Slab Prior - George and McCulloch 1992

In [ ]:
with pm.Model() as model2:
    # Spike and Slab Priors
    spike_prior = pm.Normal.dist(mu=0, sigma=0.01)
    slab_prior = pm.Normal.dist(mu=0, sigma=100)
    
    # w = pm.Bernoulli("w", p=0.25, shape=k) # Multinomial
    w = pm.Beta("w", alpha=0.25, beta=0.75, shape=k)
    weights = pm.Deterministic("weights", pm.math.stack([1.-w, w], axis=1))

    # Independent Normal Priors
    beta = pm.Mixture("beta", w=weights, comp_dists=[spike_prior, slab_prior], shape=k)
    sd = pm.Gamma("sd", alpha=0.05, beta=0.1)

    # Likelihood
    mu = pm.Deterministic("mu", tt.dot(x, beta))
    y_ = pm.Normal("y_", mu=mu, sd=sd, observed=y)

In [ ]:
model2.check_test_point()

In [ ]:
with model2:
    tr = pm.sample(2000, tune=1000, cores=4)

In [ ]:
print(true_beta)
summary(tr, var_names=["beta"])

In [ ]:
print(true_beta)
fig1 = plot_trace(tr, var_names=["w"])

In [ ]:
with pm.Model() as model3:
    
    # Priors
    sd = pm.Gamma("sd", alpha=0.05, beta=0.1, shape=1)
    tau = pm.HalfCauchy("tau", beta=1, shape=1) # global shrinkage
    lam = pm.HalfCauchy("lam", beta=1/s, shape=k) # local shrinkage

    scale = pm.Deterministic("scale", pm.math.sqr(lam*tau))
    beta_tilde = pm.Normal("beta_tilde", mu=0, sigma=scale, shape=k)
    beta = pm.Deterministic("beta", beta_tilde*tau*lam*sd)
    
    kappa = pm.Deterministic("kappa", 1/(1+pm.math.sqr(lam)))

    # Likelihood
    mu = pm.Deterministic("mu", tt.dot(x, beta))
    y_ = pm.Normal("y_obs", mu=mu, sd=sd, observed=y)


In [ ]:
model3.check_test_point()

In [ ]:
with model3:
    step = pm.NUTS(target_accept=0.95, max_treedepth=15, t0=15)
    tr = pm.sample(1000, tune=1000, cores=4, chains=8, step=step)

In [ ]:
summary(tr, var_names=["kappa", "beta", "sd"])

In [ ]:
lines= [("beta", {"beta_dim_0": i}, true_beta[i]) for i in range(0, k)]
fig1 = plot_trace(tr, var_names=["beta"], lines=lines)

### Finnish Horseshoe Prior - Piironen and Vehtari 2017

In [ ]:
# Hyperparameters
m0 = p  # expected number of nonzero predictors
M = k
N = n
sigma = np.var(y)
tau0 = m0/(M-m0)*sigma/np.sqrt(N)

In [ ]:
nu = 25.
s_sq = 1

with pm.Model() as model4:
    
    # Priors
    sd = pm.Gamma("sd", alpha=0.05, beta=0.1, shape=1)
    # sd = pm.Normal("sd", mu=0.05, sd=2)

    # Hyperpriors

    tau = pm.HalfCauchy("tau", beta=tau0, shape=1) 
    c_sq = pm.InverseGamma("c_sq", alpha=nu/2, beta=nu/2*s_sq, shape=1)
    lam = pm.HalfCauchy("lam", beta=1, shape=k)     
    lam_tilde = pm.Deterministic("lam_tilde", pm.math.sqrt(c_sq)*lam/pm.math.sqrt(c_sq + pm.math.sqr(tau*lam)))

    beta_tilde = pm.Normal("beta_tilde", mu=0, sd=1, shape=k)
    beta = pm.Deterministic("beta", tau*lam_tilde*beta_tilde)
    
    # Likelihood
    mu = pm.Deterministic("mu", tt.dot(x, beta))
    y_ = pm.Normal("y_obs", mu=mu, sd=sd, observed=y)


In [ ]:
model4.check_test_point()

In [ ]:
with model4:
    tr = pm.sample(2000, tune=1000)

In [ ]:
print(true_beta)
summary(tr, var_names=["beta", "sd"])

In [ ]:
print(true_beta)
fig1 = plot_trace(tr, var_names=["beta"])

## Logistic Regression



In [ ]:
# Simulate Data
k = 20
n = 1000
sigma = 0.5

np.random.seed(615)
true_beta = np.random.randn(k)
true_beta[[2 ,3, 5, 7, 8, 9, 10, 11, 12, 13,  15, 16, 17, 19]] = 0

L = np.array(
    [[1., 0., 0., 0., 0., 0.],
    [0.5, 0.4, 0., 0., 0., 0.],
    [0.2, 0.5 , -0.7, 0., 0., 0.],
    [0., 0., 0., 1., 0., 0.],
    [0.7, 0.06, 0.03, 0., 0.8, 0.],
    [-0.7, 0., 0., 0., 0.08, .1]]
    )
K = L@L.T

# Predictor variable
#X = pm.MvNormal.dist(mu=np.zeros(6), cov=K, shape=[1, 6]).random(size=n)
X = pm.MvNormal.dist(mu=np.zeros(k), cov=np.diag(np.ones(k)), shape=[1, k]).random(size=n)
x = _shared(X)

# Simulate outcome variable
y = X.dot(true_beta) + np.random.randn(n) * sigma 


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
plt.hist(y)

In [ ]:
z = np.zeros(y.shape)

In [ ]:
z[np.where(y > 0)] = 1

In [ ]:
np.var(y)

In [ ]:
# Hyperparameters
m0 = k -2  # expected number of relevant slopes
M = k
N = n
sigma = 1/(np.mean(y)*(1-np.mean(y)))
tau0 = m0/(M-m0)*sigma/np.sqrt(N)

In [ ]:
# Hyperparameters
m0 = k -2  # expected number of relevant slopes
M = k
N = n
sigma = 1/(np.mean(y)*(1-np.mean(y)))
tau0 = m0/(M-m0)*sigma/np.sqrt(N)
nu = 25.
s_sq = 1

with pm.Model() as logit:
    
    # Priors
    sd = pm.Gamma("sd", alpha=0.05, beta=0.1, shape=1)
    # sd = pm.Normal("sd", mu=0.05, sd=2)

    # Hyperpriors
    tau = pm.HalfCauchy("tau", beta=tau0, shape=1) 
    c_sq = pm.InverseGamma("c_sq", alpha=nu/2, beta=nu/2*s_sq, shape=1)
    lam = pm.HalfCauchy("lam", beta=1, shape=k)     
    lam_tilde = pm.Deterministic("lam_tilde", pm.math.sqrt(c_sq)*lam/pm.math.sqrt(c_sq + pm.math.sqr(tau*lam)))

    beta_tilde = pm.Normal("beta_tilde", mu=0, sd=1, shape=k)
    beta = pm.Deterministic("beta", tau*lam_tilde*beta_tilde)
    
    # Likelihood
    mu = pm.Deterministic("mu", tt.dot(x, beta))
    p = pm.Deterministic('p', pm.math.exp(pm.Normal.dist(0, 1).logcdf(mu)))
    y_ = pm.Bernoulli('y', p=p, observed=z)


In [ ]:
logit.check_test_point()

In [ ]:
with logit:
    tr = pm.sample(2000, tune=1000)

In [ ]:
print(true_beta)

In [ ]:
summary(tr, var_names=["beta", "sd"])

In [ ]:
fig = plot_trace(tr, var_names=["beta"], )

In [ ]:
with logit:
    ppc = pm.sample_posterior_predictive(tr, samples=2000)

In [ ]:
z_pred_samps = ppc['y']
z_pred = np.zeros(z_pred_samps.shape[1])
z_probs = np.zeros([z_pred.shape[0], 2])

for i in range(0, len(y)):

    p1 = np.mean(z_pred_samps[:,i] == 0)
    p2 = np.mean(z_pred_samps[:,i] == 1)
    probs = [p1, p2]
    z_probs[i] = probs
    z_pred[i] = probs.index(max(probs))

In [ ]:
print(confusion_matrix(z, z_pred))

In [ ]:
print(f"accuracy:{round(accuracy_score(y_true=z, y_pred=z_pred),2)}")